# Import translated datasets

In [5]:
import pandas as pd
tw = pd.read_csv("../data/processed/Translated/twitter/dataset.csv")
sptr = pd.read_csv("../data/processed/Translated/SentiPolc/training_set_sentipolc16.csv")

#defining gold standard dataframes to get final results
gs1 = pd.read_csv("../data/processed/Translated/twitter/dataset.csv")
gs2 = pd.read_csv("../data/processed/Translated/SentiPolc/training_set_sentipolc16.csv")

# PreProcessing

## Enti Pubblici

In [6]:
from nltk.corpus import stopwords

#Keep original tweets
tw['tweetOrig_eng']=tw['eng']

#Adding a space before and after each stopword in such a way not to consider the case in which the stopword is contained in a word
words = set(stopwords.words('english'))
stopwords = [' ' + x + ' ' for x in words]

emoticons = ["😀","😃","😄","😁","😆","🤣","😂","🙂","😊","😍","🥰","🤩","☺","🥳","😒","😔","😟","🙁","☹","😥","😢","😭","😱","😞","😓","😩","😫","😡","😠","🤬"]

tw.eng = tw.eng.replace("@[\w]*[_-]*[\w]*"," ",regex=True)   # tag removal
tw.eng = tw.eng.replace("https?://[\w/%-.]*"," ",regex=True) # Url removal
# Removing everything except the letters of the alphabet and the emoticons
tw.tweet_x = tw.tweet_x.replace('[^ a-zA-Zà-ú'
                            '\😀\😃\😄\😁\😆\🤣\😂\🙂\😊\😍\🥰\🤩\☺\🥳\😒\😔\😟\🙁\☹\😥\😭\😱\😞\😓\😩\😫\😡\😠\🤬]', " ",regex=True)    
for word in emoticons:
    tw.eng = tw.eng.replace(word, " "+word+" ",regex=True) 

tw.eng = tw.eng.replace('\s+', ' ',regex=True)               # Removal of excess spaces
tw.eng = tw.eng.replace('^ ', '', regex=True)                # Removing the space at the beginning
tw.eng = tw.eng.replace(' $', '', regex=True)                # Removing the space at the end
tw.eng = tw.eng.apply(lambda x: x.lower())                   # Making everything in lowercase
tw.eng = tw.eng.replace('^', ' ', regex=True) 
tw.eng = tw.eng.replace('$', ' ', regex=True)

for word in stopwords:
    tw.eng = tw.eng.replace(word, ' ',regex=True)

# Removing the spaces at the beginning and at the end of every tweet
tw.eng = tw.eng.apply(lambda x: x.strip())
# Removing the empty tweets
tw = tw[tw.eng != '']

## SentiPolc

In [9]:
from nltk.corpus import stopwords

#Keep original tweets
sptr['tweetOrig_eng']=sptr['eng']

#Adding a space before and after each stopword in such a way not to consider the case in which the stopword is contained in a word
words = set(stopwords.words('english'))
stopwords = [' ' + x + ' ' for x in words]

emoticons = ["😀","😃","😄","😁","😆","🤣","😂","🙂","😊","😍","🥰","🤩","☺","🥳","😒","😔","😟","🙁","☹","😥","😢","😭","😱","😞","😓","😩","😫","😡","😠","🤬"]

sptr.eng = sptr.eng.replace("@[\w]*[_-]*[\w]*"," ",regex=True)   # tag removal
sptr.eng = sptr.eng.replace("https?://[\w/%-.]*"," ",regex=True) # Url removal
# Removing everything except the letters of the alphabet and the emoticons
sptr.eng = sptr.eng.replace('[^ a-zA-Zà-ú'
                            '\😀\😃\😄\😁\😆\🤣\😂\🙂\😊\😍\🥰\🤩\☺\🥳\😒\😔\😟\🙁\☹\😥\😭\😱\😞\😓\😩\😫\😡\😠\🤬]', " ",regex=True)    
for word in emoticons:
    sptr.eng = sptr.eng.replace(word, " "+word+" ",regex=True) 

sptr.eng = sptr.eng.replace('\s+', ' ',regex=True)               # Removal of excess spaces
sptr.eng = sptr.eng.replace('^ ', '', regex=True)                # Removing the space at the beginning
sptr.eng = sptr.eng.replace(' $', '', regex=True)                # Removing the space at the end
sptr.eng = sptr.eng.apply(lambda x: x.lower())                   # Making everything in lowercase
sptr.eng = sptr.eng.replace('^', ' ', regex=True) 
sptr.eng = sptr.eng.replace('$', ' ', regex=True)

for word in stopwords:
    sptr.eng = sptr.eng.replace(word, ' ',regex=True)

# Removing the spaces at the beginning and at the end of every tweet
sptr.eng = sptr.eng.apply(lambda x: x.strip())
# Removing the empty tweets
sptr = sptr[sptr.eng != '']

## Defining list of tweets

In [10]:
tw = tw['eng']
sptr = sptr['eng']

## Convert tweets to txt files to submit to SentiStrenght

In [12]:
tw.to_csv("../data/external/SentiStrenght/toClassify/translated_tw.txt",header=None,index=None,sep=' ',mode='a')
sptr.to_csv("../data/external/SentiStrenght/toClassify/translated_sptr.txt",header=None,index=None,sep=' ',mode='a')

# SentiStrenght

In [13]:
import subprocess
import shlex
import os.path
import sys

In [14]:
SentiStrengthLocation = "C:/Users/polic/Desktop/Data Science/Tesi/Risorse/SentiStrength/SentiStrengthCom.jar" # path for .jar file
SentiStrengthLanguageFolder = "../data/external/SentiStrenght/SentStrength_Data" #path for directory containing SentiStrenght support files

In [15]:
#Test code
if not os.path.isfile(SentiStrengthLocation):
    print("SentiStrength not found at: ", SentiStrengthLocation)
if not os.path.isdir(SentiStrengthLanguageFolder):
    print("SentiStrength data folder not found at: ", SentiStrengthLanguageFolder)

In [16]:
def RateSentiment(sentiString):
    #open a subprocess using shlex to get the command line string into the correct args list format
    p = subprocess.Popen(shlex.split("java -jar '" + SentiStrengthLocation + "' stdin sentidata '" + SentiStrengthLanguageFolder + "'"),stdin=subprocess.PIPE,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    #communicate via stdin the string to be rated. Note that all spaces are replaced with +
    b = bytes(sentiString.replace(" ","+"), 'utf-8') #Can't send string in Python 3, must send bytes
    stdout_byte, stderr_text = p.communicate(b)
    stdout_text = stdout_byte.decode("utf-8")  #convert from byte
    stdout_text = stdout_text.rstrip().replace("/t"," ") #remove the tab spacing between the positive and negative ratings. e.g. 1    -5 -> 1 -5
    return stdout_text + " " + sentiString

## SentiStrenght from file

# Translated tweets

In [17]:
FileToClassify = "../data/external/SentiStrenght/toClassify/translated_tw.txt"
if not os.path.isfile(FileToClassify):
    print("File to classify not found at: ", FileToClassify)

In [ ]:
print("Running SentiStrength on file " + FileToClassify + " with command:")
cmd = 'java -jar "' + SentiStrengthLocation + '" sentidata "' + SentiStrengthLanguageFolder + '" input "' + FileToClassify + '"'
print(cmd)
p = subprocess.Popen(shlex.split(cmd),stdin=subprocess.PIPE,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
classifiedSentimentFile = os.path.splitext(FileToClassify)[0] + "0_out.txt"
print("Finished! The results will be in:/n" + classifiedSentimentFile)

## Results

In [21]:
tw_out = pd.read_csv("../data/external/SentiStrenght/output/translated_tw0_out.txt",sep='\t',encoding='latin-1')
tw_out.head()

,Positive,Negative,Text
0,2,-1,new #bonus #inps coming! find get 1000 euros! ...
1,3,-2,"so, let's recap; survivor's pension granted po..."
2,2,-1,"dear dear accounts well, know worth ð? ð? ..."
3,1,-1,"photo - #napoli, opening #voragine, forty peop..."
4,2,-1,maxi #protezionecivile tutorial. scenario bosc...


In [22]:
#Reset index to join dataframes
tw_out=tw_out.reset_index()
gs1=gs1.reset_index()
#Mapping SentiStrenght scores to sentiment classes
tw_out.loc[tw_out['Positive'] > 1, 'Pos_SS'] = 'yes'
tw_out.loc[tw_out['Negative'] < -1, 'Neg_SS'] = 'yes'
tw_out.loc[(tw_out['Positive'] <= 1)&(tw_out['Negative'] >= -1) , 'Neut_SS'] = 'yes'

#Mapping GoldStandard scores to sentiment classes
gs1.loc[gs1['Class'] =='pos', 'Pos_GS'] = 'yes'
gs1.loc[gs1['Class'] == 'neg', 'Neg_GS'] = 'yes'
gs1.loc[gs1['Class'] == 'neut', 'Neut_GS'] = 'yes'
gs1.loc[gs1['Class'] == 'mix', 'Pos_GS'] = 'yes'
gs1.loc[gs1['Class'] == 'mix', 'Neg_GS'] = 'yes'

#Join dataframes
val = gs1.merge(tw_out, how='inner', on='index')
val = val.fillna("no")
val = val[['tweet_id','year','month','day','tweet_x','tweetOrig','Pos_GS','Pos_SS', 'Neg_GS','Neg_SS', 'Neut_GS','Neut_SS', 'Irony']]

## Metrics
### Positive

In [23]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

print("P: ",precision_score(val['Pos_GS'],val['Pos_SS'],labels=['yes','no'],pos_label='yes'))
print("R: ",recall_score(val['Pos_GS'],val['Pos_SS'],labels=['yes','no'],pos_label='yes'))
print("F: ",f1_score(val['Pos_GS'],val['Pos_SS'],labels=['yes','no'],pos_label='yes'))

P:  0.17493796526054592
R:  0.734375
F:  0.28256513026052105


### Negative

In [24]:
print("P: ",precision_score(val['Neg_GS'],val['Neg_SS'],labels=['yes','no'],pos_label='yes'))
print("R: ",recall_score(val['Neg_GS'],val['Neg_SS'],labels=['yes','no'],pos_label='yes'))
print("F: ",f1_score(val['Neg_GS'],val['Neg_SS'],labels=['yes','no'],pos_label='yes'))

P:  0.6052631578947368
R:  0.4951560818083961
F:  0.5447010065127293


### Neutral

In [25]:
print("P: ",precision_score(val['Neut_GS'],val['Neut_SS'],labels=['yes','no'],pos_label='yes'))
print("R: ",recall_score(val['Neut_GS'],val['Neut_SS'],labels=['yes','no'],pos_label='yes'))
print("F: ",f1_score(val['Neut_GS'],val['Neut_SS'],labels=['yes','no'],pos_label='yes'))

P:  0.5387045813586098
R:  0.42098765432098767
F:  0.4726264726264726


# Translated SentiPolc

In [26]:
FileToClassify = "../data/external/SentiStrenght/toClassify/translated_sptr.txt"
if not os.path.isfile(FileToClassify):
    print("File to classify not found at: ", FileToClassify)

In [ ]:
print("Running SentiStrength on file " + FileToClassify + " with command:")
cmd = 'java -jar "' + SentiStrengthLocation + '" sentidata "' + SentiStrengthLanguageFolder + '" input "' + FileToClassify + '"'
print(cmd)
p = subprocess.Popen(shlex.split(cmd),stdin=subprocess.PIPE,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
classifiedSentimentFile = os.path.splitext(FileToClassify)[0] + "0_out.txt"
print("Finished! The results will be in:/n" + classifiedSentimentFile)

## Results

In [27]:
sptr_out = pd.read_csv("../data/external/SentiStrenght/output/translated_sptr0_out.txt",sep='\t',encoding='latin-1')
sptr_out.head()

,Positive,Negative,Text
0,1,-2,meanwhile game via nazionale becomes complicat...
1,1,-3,false illusions unpleasant realities mario mon...
2,1,-3,false illusions unpleasant realities editorial...
3,1,-2,mario monti berlusconi spare italy blame causi...
4,1,-2,mario monti berlusconi spare italy blame causi...


In [28]:
#Reset index to join dataframes
sptr_out=sptr_out.reset_index()
gs2=gs2.reset_index()
#Mapping SentiStrenght scores to sentiment classes
sptr_out.loc[sptr_out['Positive'] > 1, 'Pos_SS'] = 'yes'
sptr_out.loc[sptr_out['Negative'] < -1, 'Neg_SS'] = 'yes'
sptr_out.loc[(sptr_out['Positive'] <= 1)&(sptr_out['Negative'] >= -1) , 'Neut_SS'] = 'yes'

#Mapping SentiPolc scores to sentiment classes
gs2.loc[gs2['opos'] ==1, 'Pos_GS'] = 'yes'
gs2.loc[gs2['oneg'] ==1, 'Neg_GS'] = 'yes'
gs2.loc[(gs2['opos'] ==0) & (gs2['oneg'] ==0), 'Neut_GS'] = 'yes'
gs2 = gs2.fillna("no")

#Join dataframes
val = gs2.merge(sptr_out, how='inner', on='index')
val = val.fillna("no")
val = val[['idtwitter','text','eng','Pos_GS','Pos_SS', 'Neg_GS','Neg_SS', 'Neut_GS','Neut_SS']]

## Metrics
### Positive

In [29]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

print(precision_score(val['Pos_GS'],val['Pos_SS'],labels=['yes','no'],pos_label='yes'))
print(recall_score(val['Pos_GS'],val['Pos_SS'],labels=['yes','no'],pos_label='yes'))
print(f1_score(val['Pos_GS'],val['Pos_SS'],labels=['yes','no'],pos_label='yes'))


0.4090751645306547
0.5772238514173998
0.4788161362254207


### Negative

In [30]:
print(precision_score(val['Neg_GS'],val['Neg_SS'],labels=['yes','no'],pos_label='yes'))
print(recall_score(val['Neg_GS'],val['Neg_SS'],labels=['yes','no'],pos_label='yes'))
print(f1_score(val['Neg_GS'],val['Neg_SS'],labels=['yes','no'],pos_label='yes'))

0.5638562789043046
0.5315224681421864
0.547212152597963


### Neutral

In [31]:
print(precision_score(val['Neut_GS'],val['Neut_SS'],labels=['yes','no'],pos_label='yes'))
print(recall_score(val['Neut_GS'],val['Neut_SS'],labels=['yes','no'],pos_label='yes'))
print(f1_score(val['Neut_GS'],val['Neut_SS'],labels=['yes','no'],pos_label='yes'))

0.5148176291793313
0.4811789772727273
0.49743024963289284
